<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">How to use SingleStore with Spark ML for Fraud Detection</h1>
    </div>
</div>

In [3]:
!pip cache purge --quiet

In [4]:
!conda install -y --quiet -c conda-forge openjdk

Solving environment: ...working... done

# All requested packages already installed.



In [5]:
!pip install pyspark --quiet

In [6]:
import os
import pandas as pd
import requests

from pyspark.sql import SparkSession
from singlestoredb import notebook as nb
from singlestoredb.management import get_secret

In [7]:
os.makedirs("jars", exist_ok = True)

In [8]:
def download_jar(url, destination):
    response = requests.get(url)
    with open(destination, "wb") as f:
        f.write(response.content)

jar_urls = [
    ("https://repo1.maven.org/maven2/com/singlestore/singlestore-jdbc-client/1.2.4/singlestore-jdbc-client-1.2.4.jar", "jars/singlestore-jdbc-client-1.2.4.jar"),
    ("https://repo1.maven.org/maven2/com/singlestore/singlestore-spark-connector_2.12/4.1.8-spark-3.5.0/singlestore-spark-connector_2.12-4.1.8-spark-3.5.0.jar", "jars/singlestore-spark-connector_2.12-4.1.8-spark-3.5.0.jar"),
    ("https://repo1.maven.org/maven2/org/apache/commons/commons-dbcp2/2.12.0/commons-dbcp2-2.12.0.jar", "jars/commons-dbcp2-2.12.0.jar"),
    ("https://repo1.maven.org/maven2/org/apache/commons/commons-pool2/2.12.0/commons-pool2-2.12.0.jar", "jars/commons-pool2-2.12.0.jar"),
    ("https://repo1.maven.org/maven2/io/spray/spray-json_3/1.3.6/spray-json_3-1.3.6.jar", "jars/spray-json_3-1.3.6.jar"),
    ("https://repo1.maven.org/maven2/com/mysql/mysql-connector-j/9.0.0/mysql-connector-j-9.0.0.jar", "jars/mysql-connector-j-9.0.0.jar")
]

for url, destination in jar_urls:
    download_jar(url, destination)

print("JAR files downloaded successfully")

JAR files downloaded successfully


In [9]:
# Create a Spark session
spark = (SparkSession
             .builder
             .config("spark.jars", ",".join([destination for _, destination in jar_urls]))
             .appName("Spark Fraud Detection")
             .getOrCreate()
        )

spark.sparkContext.setLogLevel("ERROR")

24/10/13 11:24:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [11]:
pandas_df = pd.read_csv(
    nb.stage.open("creditcard.csv", "r")
)

In [12]:
df = spark.createDataFrame(pandas_df)

In [13]:
df.count()

284807

In [14]:
df.groupBy("Class").count().show()

+-----+------+
|Class| count|
+-----+------+
|    0|284315|
|    1|   492|
+-----+------+



<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [15]:
from sqlalchemy import *

db_connection = create_engine(connection_url)
url = db_connection.url

In [16]:
password = get_secret("password")
host = url.host
port = url.port
cluster = host + ":" + str(port)

In [17]:
spark.conf.set("spark.datasource.singlestore.ddlEndpoint", cluster)
spark.conf.set("spark.datasource.singlestore.user", "admin")
spark.conf.set("spark.datasource.singlestore.password", password)
spark.conf.set("spark.datasource.singlestore.disablePushdown", "false")

In [18]:
%%time

(df.write
   .format("singlestore")
   .option("loadDataCompression", "LZ4")
   .mode("overwrite")
   .save("fraud_detection.credit_card_tx")
)

CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 5.11 s


In [19]:
jdbc_url = f"jdbc:mysql://{host}:{port}/fraud_detection"

properties = {
    "user": "admin",
    "password": password,
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [20]:
%%time

df.write.jdbc(
    url = jdbc_url,
    table = "credit_card_tx2",
    mode = "overwrite",
    properties = properties
)

CPU times: user 220 ms, sys: 10 ms, total: 230 ms
Wall time: 1min 18s


In [21]:
spark.stop()